<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-proyecto-equipo-149/blob/main/Reto_P1_AguasSubterraneas_Equipo149.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maestría en Inteligencia Artificial Aplicada**

# **Curso: Ciencia de Datos**

Tecnológico de Monterrey
Prof María de la Paz Rico

# **Reto Entrega 1: limpieza, analisis, visualización y k-means**

**Nombres y matrículas de los integrantes del equipo:**

Ignacio Valdés Reyes A01793262

Carlos Gustavo López Lerma A01793468

In [44]:
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import math #Esta libreria la usamos lara el ramsey y el Mape
import matplotlib.pyplot as plt
import requests, zipfile #Librerira para zip de nuestros origen de datos
from io import BytesIO

from imblearn.metrics import geometric_mean_score, classification_report_imbalanced
from google.colab import drive

from sklearn.model_selection import learning_curve, validation_curve
from sklearn.preprocessing import QuantileTransformer #Esta libreria la usamos al graficar datos
from sklearn.preprocessing import power_transform #esta igual
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split #Para hacer las particiones
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score         
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import  cross_validate,  RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier #Esta tambien la usamoie en el ejercicio 5e
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold

In [45]:
#Cargamos los datos de monitoreo de aguas subterraneas
url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'
req = requests.get(url)
zipfile.ZipFile(BytesIO(req.content)).extractall('unzipped_zip/')
dfsub=pd.read_csv('unzipped_zip/Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv', encoding = 'latin1')
dfsub.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


In [46]:
dfsub.describe()

,LONGITUD,LATITUD,PERIODO,ALC_mg/L,CONDUCT_mS/cm,SDT_mg/L
count,1068.000000,1068.000000,1068.0,1064.000000,1062.000000,0.0
mean,-101.891007,23.163618,2020.0,235.633759,1138.953013,NaN
std,6.703263,3.887670,0.0,116.874291,1245.563674,NaN
min,-116.664250,14.561150,2020.0,26.640000,50.400000,NaN
25%,-105.388865,20.212055,2020.0,164.000000,501.750000,NaN
50%,-102.174180,22.617190,2020.0,215.527500,815.000000,NaN
75%,-98.974716,25.510285,2020.0,292.710000,1322.750000,NaN
max,-86.864120,32.677713,2020.0,1650.000000,18577.000000,NaN


In [47]:
dfsub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CLAVE                 1068 non-null   object 
 1   SITIO                 1068 non-null   object 
 2   ORGANISMO_DE_CUENCA   1068 non-null   object 
 3   ESTADO                1068 non-null   object 
 4   MUNICIPIO             1068 non-null   object 
 5   ACUIFERO              1068 non-null   object 
 6   SUBTIPO               1068 non-null   object 
 7   LONGITUD              1068 non-null   float64
 8   LATITUD               1068 non-null   float64
 9   PERIODO               1068 non-null   int64  
 10  ALC_mg/L              1064 non-null   float64
 11  CALIDAD_ALC           1064 non-null   object 
 12  CONDUCT_mS/cm         1062 non-null   float64
 13  CALIDAD_CONDUC        1062 non-null   object 
 14  SDT_mg/L              0 non-null      float64
 15  SDT_M_mg/L           

In [61]:
dfsub.columns

Index(['CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO',
       'ACUIFERO', 'SUBTIPO', 'LONGITUD', 'LATITUD', 'PERIODO', 'ALC_mg/L',
       'CALIDAD_ALC', 'CONDUCT_mS/cm', 'CALIDAD_CONDUC', 'SDT_mg/L',
       'SDT_M_mg/L', 'CALIDAD_SDT_ra', 'CALIDAD_SDT_salin', 'FLUORUROS_mg/L',
       'CALIDAD_FLUO', 'DUR_mg/L', 'CALIDAD_DUR', 'COLI_FEC_NMP/100_mL',
       'CALIDAD_COLI_FEC', 'N_NO3_mg/L', 'CALIDAD_N_NO3', 'AS_TOT_mg/L',
       'CALIDAD_AS', 'CD_TOT_mg/L', 'CALIDAD_CD', 'CR_TOT_mg/L', 'CALIDAD_CR',
       'HG_TOT_mg/L', 'CALIDAD_HG', 'PB_TOT_mg/L', 'CALIDAD_PB', 'MN_TOT_mg/L',
       'CALIDAD_MN', 'FE_TOT_mg/L', 'CALIDAD_FE', 'SEMAFORO', 'CONTAMINANTES',
       'CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra',
       'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR',
       'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD',
       'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN',
       'CUMPLE_CON_FE'],
      dtype='

In [48]:
dfsub.isna().sum().sort_values(ascending=False) 

SDT_mg/L                1068
CONTAMINANTES            434
CALIDAD_CONDUC             6
CONDUCT_mS/cm              6
ALC_mg/L                   4
CALIDAD_ALC                4
CALIDAD_SDT_ra             2
SDT_M_mg/L                 2
CALIDAD_SDT_salin          2
CALIDAD_N_NO3              1
CALIDAD_DUR                1
N_NO3_mg/L                 1
DUR_mg/L                   1
CUMPLE_CON_COND            0
CUMPLE_CON_ALC             0
SEMAFORO                   0
CALIDAD_FE                 0
FE_TOT_mg/L                0
CALIDAD_MN                 0
CUMPLE_CON_SDT_ra          0
CUMPLE_CON_SDT_salin       0
CLAVE                      0
CUMPLE_CON_FLUO            0
CUMPLE_CON_DUR             0
CALIDAD_PB                 0
CUMPLE_CON_CF              0
CUMPLE_CON_NO3             0
CUMPLE_CON_AS              0
CUMPLE_CON_CD              0
CUMPLE_CON_CR              0
CUMPLE_CON_HG              0
CUMPLE_CON_PB              0
CUMPLE_CON_MN              0
MN_TOT_mg/L                0
CD_TOT_mg/L   

El archivo de aguas contiene 1068 registros y 57 columnas. La mayoría de las columnas no tienen ningún registro nulo o como máximo 6, esto es, máximo 0,5% de sus datos nulos lo cual muestra, por lo pronto, buena calidad de los datos. Estos datos nulos, considerando que su cantidad no es representativa, podríamos proceder a eliminarlos, sin embargo, analizaremos cada uno de estos atributos para decidir si lo más convneiente es en cambio, aplicar otra estrategia como por ejemplo la asignación del valor más frecuente, el promedio, o la copia del valor de la fila anterior.

Como excepción a lo anteior, hay dos columnas, SDT_mg/L (Solidos Disueltos Totales), que tiene todos los datos nulos, y Contaminantes.

La columna CONTAMINANTES tiene datos en  solo 634 de los 1068 registros, lo cual se interpreta no como mala calidad de los datos, sino que los datos nulos significan que estos puntos no presentan contaminantes en el agua. Se procederá en este caso a reemplazar los nulos por 'NO' (indicando que no tiene contaminantes)

La columna SDT_mg/L no tiene ningun datos, procederemos a eliminarla. 

In [70]:
#Generamos una copia de los datos para proceder a realizar las transformaciones transformaciones
dfsub2 = dfsub.copy()

In [71]:
#PASO 1
#Eliminamos la columna SDT_mg/L
dfsub2 = dfsub2.drop(axis = 1, columns = ['SDT_mg/L'])

In [72]:
#Analisis de CONTAMINANTES
print (dfsub2['CONTAMINANTES'].value_counts())

FLUO,                                            78
DT,                                              65
FLUO,AS,                                         51
CF,                                              31
AS,                                              31
                                                 ..
ALC,CONDUC,SDT_ra,SDT_salin,DT,NO3,               1
ALC,CONDUC,SDT_ra,SDT_salin,FLUO,DT,AS,MN,FE,     1
PB,MN,FE,                                         1
ALC,AS,FE,                                        1
ALC,DT,NO3,                                       1
Name: CONTAMINANTES, Length: 126, dtype: int64


In [73]:
#PASO 2
#Cambiamos los datos nulos por NO en la columna CONTAMINANTES
dfsub2.CONTAMINANTES.fillna('NO', inplace = True)

In [74]:
#Validamos que el cambio alla sido correctamente aplicado
print (dfsub2['CONTAMINANTES'].value_counts())

NO                                               434
FLUO,                                             78
DT,                                               65
FLUO,AS,                                          51
CF,                                               31
                                                ... 
ALC,FLUO,AS,FE,                                    1
ALC,CONDUC,SDT_ra,SDT_salin,FLUO,DT,AS,MN,FE,      1
PB,MN,FE,                                          1
ALC,AS,FE,                                         1
ALC,DT,NO3,                                        1
Name: CONTAMINANTES, Length: 127, dtype: int64


In [75]:
dfsub2.isna().sum().sort_values(ascending=False) 

CALIDAD_CONDUC          6
CONDUCT_mS/cm           6
ALC_mg/L                4
CALIDAD_ALC             4
CALIDAD_SDT_ra          2
SDT_M_mg/L              2
CALIDAD_SDT_salin       2
CALIDAD_N_NO3           1
CALIDAD_DUR             1
N_NO3_mg/L              1
DUR_mg/L                1
CUMPLE_CON_ALC          0
CUMPLE_CON_COND         0
CONTAMINANTES           0
SEMAFORO                0
CALIDAD_FE              0
FE_TOT_mg/L             0
CALIDAD_MN              0
CUMPLE_CON_SDT_ra       0
CUMPLE_CON_SDT_salin    0
CLAVE                   0
CUMPLE_CON_FLUO         0
CUMPLE_CON_DUR          0
CALIDAD_PB              0
CUMPLE_CON_CF           0
CUMPLE_CON_NO3          0
CUMPLE_CON_AS           0
CUMPLE_CON_CD           0
CUMPLE_CON_CR           0
CUMPLE_CON_HG           0
CUMPLE_CON_PB           0
CUMPLE_CON_MN           0
MN_TOT_mg/L             0
CALIDAD_CD              0
PB_TOT_mg/L             0
CALIDAD_HG              0
ORGANISMO_DE_CUENCA     0
ESTADO                  0
MUNICIPIO   

In [76]:
#Analisis de atributos con entre 1 y 6 registros nulos
print ('***CALIDAD_CONDUC***')
print (dfsub2['CALIDAD_CONDUC'].value_counts())
print ('***CONDUCT_mS/cm***')
print (dfsub2['CONDUCT_mS/cm'].value_counts())
print ('***ALC_mg/L***')
print (dfsub2['ALC_mg/L'].value_counts())
print ('***CALIDAD_ALC***')
print (dfsub2['CALIDAD_ALC'].value_counts())
print ('***CALIDAD_SDT_ra***')
print (dfsub2['CALIDAD_SDT_ra'].value_counts())
print ('***SDT_M_mg/L***')
print (dfsub2['SDT_M_mg/L'].value_counts())
print ('***CALIDAD_SDT_salin***')
print (dfsub2['CALIDAD_SDT_salin'].value_counts())
print ('***CALIDAD_N_NO3***')
print (dfsub2['CALIDAD_N_NO3'].value_counts())
print ('***CALIDAD_DUR***')
print (dfsub2['CALIDAD_DUR'].value_counts())
print ('***N_NO3_mg/L***')
print (dfsub2['N_NO3_mg/L'].value_counts())
#print(dfsub2['N_NO3_mg/L'].min(), dfsub2['N_NO3_mg/L'].max())
print ('***DUR_mg/L***')
print (dfsub2['DUR_mg/L'].value_counts())
#print(dfsub2['DUR_mg/L'].min(), dfsub2['DUR_mg/L'].max())


***CALIDAD_CONDUC***
Permisible para riego    460
Buena para riego         434
Dudosa para riego         72
Indeseable para riego     51
Excelente para riego      45
Name: CALIDAD_CONDUC, dtype: int64
***CONDUCT_mS/cm***
777.0    6
300.0    4
412.0    4
454.0    4
308.0    4
        ..
826.0    1
876.0    1
373.0    1
733.0    1
817.0    1
Name: CONDUCT_mS/cm, Length: 801, dtype: int64
***ALC_mg/L***
157.620    5
193.815    4
195.360    4
204.765    4
257.850    4
          ..
341.000    1
151.000    1
106.000    1
99.000     1
256.000    1
Name: ALC_mg/L, Length: 816, dtype: int64
***CALIDAD_ALC***
Alta                    794
Media                   187
Indeseable como FAAP     59
Baja                     24
Name: CALIDAD_ALC, dtype: int64
***CALIDAD_SDT_ra***
Excelente para riego            491
Cultivos sensibles              343
Cultivos con manejo especial    161
Cultivos tolerantes              64
Indeseable para riego             7
Name: CALIDAD_SDT_ra, dtype: int64
***SDT_M_mg/L

Tras analizar la composición de los datos en estos atributos con pocos nulos, se decide, en las variables categóricas, reemplazar los nulos por los valores más frecuentes.

In [79]:
#PASO 3
#Cambiamos los datos nulos por los valores más frecuentes en los atributos categóricos y ordinales con pocos nulos
dfsub2.CALIDAD_CONDUC.fillna('Permisible para riego', inplace = True)
dfsub2.CALIDAD_ALC.fillna('Alta', inplace = True)
dfsub2.CALIDAD_SDT_ra.fillna('Excelente para riego', inplace = True)
dfsub2.CALIDAD_SDT_salin.fillna('Potable - Dulce', inplace = True)
dfsub2.CALIDAD_N_NO3.fillna('Potable - Excelente', inplace = True)
dfsub2.CALIDAD_DUR.fillna('Potable - Dura', inplace = True)

#Eliminamos los datos nulos de los atributos numéricos dada la baja cantidad de registros
dfsub2.dropna(axis=0, inplace = True)

In [80]:
dfsub2.isna().sum().sort_values(ascending=False) 

CLAVE                   0
SITIO                   0
CALIDAD_CR              0
HG_TOT_mg/L             0
CALIDAD_HG              0
PB_TOT_mg/L             0
CALIDAD_PB              0
MN_TOT_mg/L             0
CALIDAD_MN              0
FE_TOT_mg/L             0
CALIDAD_FE              0
SEMAFORO                0
CONTAMINANTES           0
CUMPLE_CON_ALC          0
CUMPLE_CON_COND         0
CUMPLE_CON_SDT_ra       0
CUMPLE_CON_SDT_salin    0
CUMPLE_CON_FLUO         0
CUMPLE_CON_DUR          0
CUMPLE_CON_CF           0
CUMPLE_CON_NO3          0
CUMPLE_CON_AS           0
CUMPLE_CON_CD           0
CUMPLE_CON_CR           0
CUMPLE_CON_HG           0
CUMPLE_CON_PB           0
CUMPLE_CON_MN           0
CR_TOT_mg/L             0
CALIDAD_CD              0
CD_TOT_mg/L             0
CALIDAD_CONDUC          0
ORGANISMO_DE_CUENCA     0
ESTADO                  0
MUNICIPIO               0
ACUIFERO                0
SUBTIPO                 0
LONGITUD                0
LATITUD                 0
PERIODO     